In [7]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score


In [12]:
import os
import shutil

data_dir = "../FF-Det/Testing"
for root, dirs, files in os.walk(data_dir):
    for dir_name in dirs:
        if dir_name == ".ipynb_checkpoints":
            dir_path = os.path.join(root, dir_name)
            print(f"Removing: {dir_path}")
            shutil.rmtree(dir_path)  # Deletes the directory and its contents

Removing: ../FF-Det/Testing/.ipynb_checkpoints
Removing: ../FF-Det/Testing/fire/.ipynb_checkpoints
Removing: ../FF-Det/Testing/nofire/.ipynb_checkpoints


In [13]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directories
data_dir = "../FF-Det/Testing"  # Set this to the parent directory of 'fire' and 'no_fire'

# Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet50 expects 224x224 images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def is_valid_file(path):
    valid_extensions = (".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".webp")
    return path.endswith(valid_extensions)

# for root, dirs, files in os.walk(data_dir):
#     print(f"Directory: {root}")
#     print(f"Subdirectories: {dirs}")
#     print(f"Files: {files}")

# Create dataset using ImageFolder
# dataset = datasets.ImageFolder(root=data_dir, transform=transform, is_valid_file=is_valid_file)

# # Dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)



In [14]:
# Split the dataset into training (75%) and validation (25%)
train_size = int(0.75 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)



In [15]:
# Model
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # Adjust for binary classification
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



/home/dbhatka/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dbhatka/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [17]:
# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20):
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss, correct_train = 0.0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct_train += torch.sum(preds == labels).item()
        
        train_loss /= len(train_loader.dataset)
        train_acc = correct_train / len(train_loader.dataset)

        # Validation phase
        model.eval()
        val_loss, correct_val = 0.0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                correct_val += torch.sum(preds == labels).item()
        
        val_loss /= len(val_loader.dataset)
        val_acc = correct_val / len(val_loader.dataset)

        # Print metrics
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.4f}")
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")
        print("-" * 30)

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)

Epoch [1/20]
Training Loss: 0.0703, Training Accuracy: 0.9789
Validation Loss: 0.1163, Validation Accuracy: 0.9789
------------------------------
Epoch [2/20]
Training Loss: 0.0511, Training Accuracy: 0.9825
Validation Loss: 0.1587, Validation Accuracy: 0.9368
------------------------------
Epoch [3/20]
Training Loss: 0.1272, Training Accuracy: 0.9579
Validation Loss: 0.2108, Validation Accuracy: 0.9684
------------------------------
Epoch [4/20]
Training Loss: 0.1522, Training Accuracy: 0.9544
Validation Loss: 0.0764, Validation Accuracy: 0.9684
------------------------------
Epoch [5/20]
Training Loss: 0.2057, Training Accuracy: 0.9404
Validation Loss: 0.1395, Validation Accuracy: 0.9474
------------------------------
Epoch [6/20]
Training Loss: 0.2352, Training Accuracy: 0.9228
Validation Loss: 0.3169, Validation Accuracy: 0.8842
------------------------------
Epoch [7/20]
Training Loss: 0.0951, Training Accuracy: 0.9789
Validation Loss: 0.0643, Validation Accuracy: 0.9789
---------

In [18]:
# Save the trained model
torch.save(model.state_dict(), "ff-det_resnet50.pth")
# print("Model saved as 'deepfire_resnet50.pth'")